In [1]:
import pandas as pd 
import numpy as np
import requests
import io
import csv
from path import Path
import psycopg2
#from config import db_password

# Expand cols
pd.set_option('display.max_columns', 80)

CREATE DATASET

In [2]:
# Connction to AWS nba database
connection = psycopg2.connect(
    host = 'nba-db.cpmpsi1pfaz0.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'caf3rac3',
    database='postgres'
    )
cursor=connection.cursor()

In [3]:
#read in data from database for All NBA table
nba_unpivoted_sql = 'SELECT * FROM public.allnba_unpivoted'
All_NBA_Unpivoted_df = pd.read_sql(nba_unpivoted_sql, con=connection).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
All_NBA_Unpivoted_df.head()

,Season,Tm,Position,PlayerName,Season_Start_Year
0,2019-20,1st,C,Anthony Davis,2019
1,2019-20,1st,F,Giannis Antetokounmpo,2019
2,2019-20,1st,F,LeBron James,2019
3,2019-20,1st,G,James Harden,2019
4,2019-20,1st,G,Luka Dončić,2019


In [4]:

#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

#make dummies for position and team (1st, 2nd, 3rd)
All_NBA_Unpivoted_df = pd.get_dummies(All_NBA_Unpivoted_df,columns=['Position','Tm']).drop(['Season','PlayerName'],axis=1)
#rename the columns (cosmetic)
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_C':'AllNBA_Center'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_F':'AllNBA_Forward'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_G':'AllNBA_Guard'})
All_NBA_Unpivoted_df.head(10)

,Season_Start_Year,PlayerName_Formatted,AllNBA_Center,AllNBA_Forward,AllNBA_Guard,Tm_1st,Tm_2nd,Tm_3rd
0,2019,Anthony Davis,1,0,0,1,0,0
1,2019,Giannis Antetokounmpo,0,1,0,1,0,0
2,2019,LeBron James,0,1,0,1,0,0
3,2019,James Harden,0,0,1,1,0,0
4,2019,Luka Doncic,0,0,1,1,0,0
5,2019,Nikola Jokic,1,0,0,0,1,0
6,2019,Kawhi Leonard,0,1,0,0,1,0
7,2019,Pascal Siakam,0,1,0,0,1,0
8,2019,Damian Lillard,0,0,1,0,1,0
9,2019,Chris Paul,0,0,1,0,1,0


In [5]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
AllNBAPlayersList

array(['Anthony Davis', 'Giannis Antetokounmpo', 'LeBron James',
       'James Harden', 'Luka Doncic', 'Nikola Jokic', 'Kawhi Leonard',
       'Pascal Siakam', 'Damian Lillard', 'Chris Paul', 'Rudy Gobert',
       'Jayson Tatum', 'Jimmy Butler', 'Ben Simmons', 'Russell Westbrook',
       'Paul George', 'Stephen Curry', 'Joel Embiid', 'Kevin Durant',
       'Kyrie Irving', 'Blake Griffin', 'Kemba Walker',
       'LaMarcus Aldridge', 'DeMar DeRozan', 'Karl-Anthony Towns',
       'Victor Oladipo', 'Isaiah Thomas', 'DeAndre Jordan',
       'Draymond Green', 'John Wall', 'DeMarcus Cousins',
       'Andre Drummond', 'Klay Thompson', 'Kyle Lowry', 'Marc Gasol',
       'Pau Gasol', 'Tim Duncan', 'Joakim Noah', 'Dwight Howard',
       'Kevin Love', 'Tony Parker', 'Al Jefferson', 'Goran Dragic',
       'Kobe Bryant', 'Carmelo Anthony', 'David Lee', 'Dwyane Wade',
       'Andrew Bynum', 'Tyson Chandler', 'Dirk Nowitzki', 'Rajon Rondo',
       'Derrick Rose', "Amar'e Stoudemire", 'Al Horford', 'Za

In [6]:
#read in data from database for season stats table
seasons_stats_sql = 'SELECT * FROM public.seasons_stats'
season_stats_df = pd.read_sql(seasons_stats_sql, con=connection)
#divid year into season start and end
season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
season_stats_df['Year_season_end'] = pd.to_numeric(season_stats_df['Year_season_end'])
season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
season_stats_df.head()


,Year_season_end,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year_season_start
0,1980,Tim Bassett,PF,28,TOT,12,0,164.0,6.7,0.419,0.000,0.441,7.1,14.0,10.6,9.7,2.2,0.0,18.1,11.6,-0.1,0.2,0.1,0.033,-3.8,0.3,-3.6,-0.1,12.0,34.0,0.353,0.0,0.0,NaN,12.0,34.0,0.353,0.353,10.0,15.0,0.667,11.0,22.0,33.0,14.0,8.0,0.0,9.0,27.0,34.0,1979
1,1980,Tim Bassett,PF,28,NJN,7,0,92.0,7.9,0.440,0.000,0.545,7.9,12.4,10.1,5.5,2.5,0.0,12.8,13.1,0.0,0.1,0.1,0.063,-3.7,0.1,-3.5,0.0,8.0,22.0,0.364,0.0,0.0,NaN,8.0,22.0,0.364,0.364,8.0,12.0,0.667,7.0,11.0,18.0,4.0,5.0,0.0,4.0,14.0,24.0,1979
2,1980,M.L. Carr,SF,29,BOS,82,7,1994.0,14.6,0.526,0.054,0.316,6.0,12.3,9.2,10.7,2.8,1.0,14.1,20.2,1.9,3.2,5.2,0.125,-0.2,1.5,1.3,1.7,362.0,763.0,0.474,12.0,41.0,0.293,350.0,722.0,0.485,0.482,178.0,241.0,0.739,106.0,224.0,330.0,156.0,120.0,36.0,143.0,214.0,914.0,1979
3,1980,Don Chaney,SG,33,BOS,60,0,523.0,7.8,0.402,0.032,0.222,6.7,8.8,7.8,9.3,2.8,1.2,13.7,18.3,-0.4,0.8,0.4,0.033,-4.0,0.9,-3.1,-0.1,67.0,189.0,0.354,1.0,6.0,0.167,66.0,183.0,0.361,0.357,32.0,42.0,0.762,31.0,42.0,73.0,38.0,31.0,11.0,33.0,80.0,167.0,1979
4,1980,Phil Chenier,SG,29,IND,23,0,380.0,8.9,0.423,0.044,0.193,2.4,7.3,4.8,15.9,1.8,1.4,15.1,17.6,-0.3,0.2,0.0,0.000,-4.3,-1.6,-5.8,-0.4,52.0,135.0,0.385,2.0,6.0,0.333,50.0,129.0,0.388,0.393,18.0,26.0,0.692,9.0,26.0,35.0,47.0,15.0,10.0,26.0,29.0,124.0,1979


In [7]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
#AllPlayersList_df.to_csv('listofplayers_test.csv')

#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic
Jayson Tatum
Ben Simmons


In [8]:
#merge allNBA status into season stats
Season_Stats_ML_DF = pd.merge(season_stats_df,All_NBA_Unpivoted_df,how='left',left_on = ['Player','Year_season_start'], right_on = ['PlayerName_Formatted','Season_Start_Year'])

In [9]:
# get rid of duplicate cols
Season_Stats_ML_DF = Season_Stats_ML_DF.drop(['Season_Start_Year','PlayerName_Formatted'],axis=1)

In [10]:
# fill in zeros for players that arent all nba
Season_Stats_ML_DF['AllNBA_Center'] = Season_Stats_ML_DF['AllNBA_Center'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Forward'] = Season_Stats_ML_DF['AllNBA_Forward'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Guard'] = Season_Stats_ML_DF['AllNBA_Guard'].fillna(int(0))
Season_Stats_ML_DF['Tm_1st'] = Season_Stats_ML_DF['Tm_1st'].fillna(int(0))
Season_Stats_ML_DF['Tm_2nd'] = Season_Stats_ML_DF['Tm_2nd'].fillna(int(0))
Season_Stats_ML_DF['Tm_3rd'] = Season_Stats_ML_DF['Tm_3rd'].fillna(int(0))

In [11]:
#get list of positions
Positions_List = Season_Stats_ML_DF['Pos'].unique()
Positions_List

array(['PF', 'SF', 'SG', 'C', 'PG', 'SF-SG', 'SG-SF', 'C-PF', 'PF-C',
       'SF-PF', 'PG-SG', 'SG-PG', 'PF-SF', 'PG-SF', 'SG-PF', 'C-SF'],
      dtype=object)

In [12]:
# create functions to determine if player is a guard forward and/or center
def is_forward(pos):
    possible_pos = [ 'SF',  'PF', 'PF-C', 'SG-SF', 'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF']
    if pos in possible_pos: isF = 1
    else: isF = 0
    return isF

def is_guard(pos):
    possible_pos = ['PG','SG','SG-SF','PG-SG','SG-PG','SF-SG','SG-PF','PG-SF']
    if pos in possible_pos: isG = 1
    else: isG = 0
    return isG

def is_center(pos):
    possible_pos = ['C','PF-C','C-PF','C-SF']
    if pos in possible_pos: isC = 1
    else: isC = 0
    return isC

In [13]:
#Create dummy columns for G, F, C
Season_Stats_ML_DF['is_Guard'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_guard(pos) )
Season_Stats_ML_DF['is_Forward'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_forward(pos) )
Season_Stats_ML_DF['is_Center'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_center(pos) )

In [14]:
#Drop the Pos Column since we have dummy columns
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Pos'],axis=1)

In [15]:
# drop year  and player name
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_end','Player'],axis=1)
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_start'],axis=1)

Season_Stats_ML_DF.columns

Index(['Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS_pct', '3PAr', 'FTr', 'ORB_pct',
       'DRB_pct', 'TRB_pct', 'AST_pct', 'STL_pct', 'BLK_pct', 'TOV_pct',
       'USG_pct', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'FG', 'FGA', 'FG_pct', '3P', '3PA', '3P_pct', '2P', '2PA', '2P_pct',
       'eFG_pct', 'FT', 'FTA', 'FT_pct', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'AllNBA_Center', 'AllNBA_Forward',
       'AllNBA_Guard', 'Tm_1st', 'Tm_2nd', 'Tm_3rd', 'is_Guard', 'is_Forward',
       'is_Center'],
      dtype='object')

In [16]:
#Creating Per Game Statistics

#Points per Game
Season_Stats_ML_DF["PPG"] = Season_Stats_ML_DF["PTS"]/Season_Stats_ML_DF["G"]
#Assits Per Game
Season_Stats_ML_DF["APG"] = Season_Stats_ML_DF["AST"]/Season_Stats_ML_DF["G"]
#Rebounds Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["TRB"]/Season_Stats_ML_DF["G"]
#Blocks Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["BLK"]/Season_Stats_ML_DF["G"]
#STLs Per Game
Season_Stats_ML_DF["SPG"] = Season_Stats_ML_DF["STL"]/Season_Stats_ML_DF["G"]

In [17]:
Season_Stats_ML_DF['3PAr'] = Season_Stats_ML_DF['3PAr'].fillna(int(0))
Season_Stats_ML_DF['3P'] = Season_Stats_ML_DF['3P'].fillna(int(0))
Season_Stats_ML_DF['3PA'] = Season_Stats_ML_DF['3PA'].fillna(int(0))
Season_Stats_ML_DF['3P_pct'] = Season_Stats_ML_DF['3P_pct'].fillna(int(0))


In [18]:
#make dummies for team
#Season_Stats_ML_DF = pd.get_dummies(Season_Stats_ML_DF,columns=['Tm'])
#I don't like this lol

In [19]:
# Fill null values with zero
for col in list(Season_Stats_ML_DF.columns):
    Season_Stats_ML_DF[col] = Season_Stats_ML_DF[col].fillna(int(0))

ALL NBA CENTER MACHINE LEARNING SECTION

In [20]:
#Create Dataset for X & y, make sure it is only centers...we don't need guard or forward data her

# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
y = y["AllNBA_Center"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

center_model = model

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix Centers")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix Centers


,Predicted 0,Predicted 1
Actual 0,922,8
Actual 1,10,14


Accuracy Score : 0.9811320754716981
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       930
         1.0       0.64      0.58      0.61        24

    accuracy                           0.98       954
   macro avg       0.81      0.79      0.80       954
weighted avg       0.98      0.98      0.98       954



In [21]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
center_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
center_feature_importance

[(0.8087160177529342, 'WS'),
 (0.06757834612663455, 'SPG'),
 (0.049650468074336526, 'RPG'),
 (0.02137077746087154, 'DBPM'),
 (0.007416305627799432, 'TOV'),
 (0.007346831193775639, 'PF'),
 (0.004888286142657465, 'Age'),
 (0.004793564351250427, 'FT'),
 (0.004192762065871251, 'USG_pct'),
 (0.0041093525253869554, 'PER'),
 (0.0035102425893115794, 'OBPM'),
 (0.0034868356434643905, 'AST'),
 (0.0021423431741189915, 'TRB_pct'),
 (0.0020477376839461233, '3P_pct'),
 (0.0015867596695141324, 'FG_pct'),
 (0.0014121765866725658, 'STL'),
 (0.0014048235453448252, '2P'),
 (0.0007171762589928053, 'TOV_pct'),
 (0.0007139326706573657, 'FG'),
 (0.0007116679584421125, 'ORB_pct'),
 (0.0007109994117456673, '3PA'),
 (0.0007094324540367717, 'FGA'),
 (0.0007051049141937059, 'DWS'),
 (6.940005534559507e-05, 'PPG'),
 (8.656062695542297e-06, '2P_pct'),
 (0.0, 'eFG_pct'),
 (0.0, 'VORP'),
 (0.0, 'TS_pct'),
 (0.0, 'TRB'),
 (0.0, 'PTS'),
 (0.0, 'OWS'),
 (0.0, 'ORB'),
 (0.0, 'MP'),
 (0.0, 'G'),
 (0.0, 'FTr'),
 (0.0, 'FT_

ALL NBA GUARD MACHINE LEARNING SECTION

In [22]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
y = y["AllNBA_Guard"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

guard_model = model

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1806,23
Actual 1,27,33


Accuracy Score : 0.9735309687665431
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1829
         1.0       0.59      0.55      0.57        60

    accuracy                           0.97      1889
   macro avg       0.79      0.77      0.78      1889
weighted avg       0.97      0.97      0.97      1889



In [23]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
guard_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
guard_feature_importance

[(0.7759513116601915, 'WS'),
 (0.0739710095216972, 'PER'),
 (0.04372431869271224, 'FT'),
 (0.029639294716146708, 'DBPM'),
 (0.008985953775399584, 'MP'),
 (0.006997114064942397, '3PA'),
 (0.005804773407212077, 'VORP'),
 (0.00532325527223, 'G'),
 (0.005304106503235808, 'TRB_pct'),
 (0.004929961177796953, '3PAr'),
 (0.004410475845463807, 'FTr'),
 (0.004271953092279772, '2P_pct'),
 (0.0033242147169170225, 'ORB_pct'),
 (0.0031768904465087236, 'Age'),
 (0.0027987072226163144, 'STL'),
 (0.0026608805127680208, 'AST_pct'),
 (0.0021194005389175304, '3P_pct'),
 (0.002115215055456435, 'FG'),
 (0.0020497595490382874, 'BPM'),
 (0.001807034656493474, 'DWS'),
 (0.0010747230748172965, 'FG_pct'),
 (0.0010291048341565336, '3P'),
 (0.0010038742051032615, 'DRB'),
 (0.0007794263040629175, '2PA'),
 (0.0007145641307300002, 'RPG'),
 (0.0007067857670203582, 'ORB'),
 (0.0007008910208045952, 'BLK'),
 (0.0006962876946122399, 'USG_pct'),
 (0.0006944942942397203, 'FT_pct'),
 (0.0006932582470374741, 'TRB'),
 (0.00069

ALL NBA FORWARD MACHINE LEARNING SECTION

In [24]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
y = y["AllNBA_Forward"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)
forward_model = model.fit(X_train_scaled, y_train)
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix Forwards")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix Forwards


,Predicted 0,Predicted 1
Actual 0,1852,20
Actual 1,20,25


Accuracy Score : 0.9791340636411059
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1872
         1.0       0.56      0.56      0.56        45

    accuracy                           0.98      1917
   macro avg       0.77      0.77      0.77      1917
weighted avg       0.98      0.98      0.98      1917



In [25]:
sum(predictions)

45.0

In [26]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
forward_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
forward_feature_importance

[(0.8267587585930831, 'PPG'),
 (0.04008021768934842, 'VORP'),
 (0.03012325719649773, 'DRB'),
 (0.01809183933673047, 'BPM'),
 (0.01266344423917953, 'USG_pct'),
 (0.012521850466100201, 'PER'),
 (0.00780060581484645, 'WS'),
 (0.006142262102626841, 'DWS'),
 (0.005033727174886673, '3P'),
 (0.004030037625616289, '3PA'),
 (0.0038162748848143088, 'AST_pct'),
 (0.0031199001585705825, 'BLK'),
 (0.003052918600582799, 'TOV_pct'),
 (0.0026594082239351797, 'BLK_pct'),
 (0.002521507078931595, 'FTr'),
 (0.002052568378280224, 'FT_pct'),
 (0.001741651040627897, '3PAr'),
 (0.001738154451508075, 'TOV'),
 (0.0017373403350013697, 'FT'),
 (0.0016123222288439848, 'eFG_pct'),
 (0.0010661710808289998, 'TRB'),
 (0.0010483603713369951, 'DBPM'),
 (0.001029940349875767, 'MP'),
 (0.0010269570023928567, 'ORB'),
 (0.0010265460440703983, 'SPG'),
 (0.00101108177187661, 'Age'),
 (0.0009713257644242679, 'RPG'),
 (0.0009611901912302887, 'FG'),
 (0.0007001753760686476, '2P'),
 (0.0006872116314980164, '3P_pct'),
 (0.00067717

Making Predictions

In [27]:
filename =  r'C:\Users\jack.getman\OneDrive - WaterBridge Resources, LLC\Desktop\DatacampNew\Bootcamp_Final_Project\2021Data\season_stats2021.csv'
seasons_stats_2021 = pd.read_csv(filename) 
try:
    seasons_stats_2021 = seasons_stats_2021.drop(['Unnamed: 0'], axis=1)
except:
    print('switch the data source')

#Creating Per Game Statistics

#Points per Game
seasons_stats_2021["PPG"] = seasons_stats_2021["PTS"]/seasons_stats_2021["G"]
#Assits Per Game
seasons_stats_2021["APG"] = seasons_stats_2021["AST"]/seasons_stats_2021["G"]
#Rebounds Per Game
seasons_stats_2021["RPG"] = seasons_stats_2021["TRB"]/seasons_stats_2021["G"]
#Blocks Per Game
seasons_stats_2021["RPG"] = seasons_stats_2021["BLK"]/seasons_stats_2021["G"]
#STLs Per Game
seasons_stats_2021["SPG"] = seasons_stats_2021["STL"]/seasons_stats_2021["G"]    
    
centers_stats_2021 = seasons_stats_2021
forwards_stats_2021 = seasons_stats_2021
guards_stats_2021 = seasons_stats_2021

#seasons_stats_2021=seasons_stats_2021[['Age','G','GS','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','STL_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','WS/48','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']]
#seasons_stats_2021




centers_stats_2021['is_Center'] = centers_stats_2021['Pos'].apply(lambda pos: is_center(pos) )
guards_stats_2021['is_Guard'] = guards_stats_2021['Pos'].apply(lambda pos: is_guard(pos) )
forwards_stats_2021['is_Forward'] = forwards_stats_2021['Pos'].apply(lambda pos: is_forward(pos) )

centers_stats_2021=centers_stats_2021[centers_stats_2021['is_Center']==1]
guards_stats_2021=guards_stats_2021[guards_stats_2021['is_Guard']==1]
forwards_stats_2021=forwards_stats_2021[forwards_stats_2021['is_Forward']==1]

#centers_stats_2021=centers_stats_2021[centers_stats_2021['is_Center']==1].reset_index()
#guards_stats_2021=guards_stats_2021[guards_stats_2021['is_Guard']==1].reset_index()
#forwards_stats_2021=forwards_stats_2021[forwards_stats_2021['is_Forward']==1].reset_index()



In [29]:
centers_stats_2021.head()

,Player,Pos,Age,Tm,G,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GS,PPG,APG,RPG,SPG,is_Center,is_Guard,is_Forward
2,Steven Adams,C,27,NOP,58,1605,15.1,0.596,0.010,0.438,14.4,20.4,17.4,9.1,1.6,2.2,17.5,11.7,2.3,1.7,4.0,0.119,-0.3,-0.5,-0.8,0.5,189,308,0.614,0,3,0.000,189,305,0.620,0.614,60,135,0.444,213,301,514,111,54,38,78,113,438,58,7.551724,1.913793,0.655172,0.931034,1,0,0
3,Bam Adebayo,C,23,MIA,64,2143,22.7,0.626,0.010,0.443,7.7,22.6,15.3,26.9,1.7,3.2,15.0,23.7,5.6,3.2,8.8,0.197,2.9,1.9,4.7,3.6,456,800,0.570,2,8,0.250,454,792,0.573,0.571,283,354,0.799,142,431,573,346,75,66,169,145,1197,64,18.703125,5.406250,1.031250,1.171875,1,0,0
4,LaMarcus Aldridge,C,35,TOT,26,674,15.7,0.556,0.270,0.159,3.0,15.8,9.4,11.0,0.8,3.7,7.9,22.2,0.5,0.6,1.1,0.080,-0.2,-0.4,-0.6,0.2,140,296,0.473,31,80,0.388,109,216,0.505,0.525,41,47,0.872,19,99,118,49,11,29,27,47,352,23,13.538462,1.884615,1.115385,0.423077,1,0,0
8,Jarrett Allen,C,22,TOT,63,1864,20.1,0.661,0.039,0.602,11.6,26.3,18.9,8.7,0.8,4.3,14.1,16.6,4.3,2.1,6.4,0.166,1.4,-0.4,0.9,1.4,298,482,0.618,6,19,0.316,292,463,0.631,0.624,204,290,0.703,196,435,631,106,32,90,100,96,806,45,12.793651,1.682540,1.428571,0.507937,1,0,0
21,Deandre Ayton,C,22,PHO,69,2115,20.3,0.653,0.029,0.252,12.4,26.3,19.5,6.8,1.0,3.4,11.9,18.2,5.3,3.1,8.4,0.191,0.9,0.0,0.9,1.6,430,687,0.626,4,20,0.200,426,667,0.639,0.629,133,173,0.769,228,499,727,99,41,81,103,196,997,69,14.449275,1.434783,1.173913,0.594203,1,0,0


In [ ]:
# we need to use the centers_stats_2021 and the center ML model to predict which are all NBA centers